In [1]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from moviepy.editor import VideoFileClip
from moviepy.video.io.VideoFileClip import VideoFileClip
#from tracker import *

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2613248/45929032 bytes (5.7%)5873664/45929032 bytes (12.8%)9060352/45929032 bytes (19.7%)12312576/45929032 bytes (26.8%)15564800/45929032 bytes (33.9%)19054592/45929032 bytes (41.5%)22331392/45929032 bytes (48.6%)25288704/45929032 bytes (55.1%)28794880/45929032 bytes (62.7%)32595968/45929032 bytes (71.0%)35078144/45929032 bytes (76.4%)38125568/45929032 bytes (83.0%)

In [2]:
def perspective_warp(img, 
                     dst_size=(1280,720),
                     src=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]),#[(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)] [(0.30,0.65),(0.60,0.65),(0.1,1),(1,1)]
                     dst=np.float32([(0,0), (1, 0), (0,1), (1,1)])):
    img_size = np.float32([(img.shape[1],img.shape[0])])
    src = src* img_size
    # For destination points, I'm arbitrarily choosing some points to be a nice fit for displaying our warped result again, not exact, but close enough for our purposes
    dst = dst * np.float32(dst_size)
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, dst_size,cv2.INTER_LINEAR)
    return warped

In [3]:
def inv_perspective_warp(img, 
                     dst_size=(1280,720),
                     src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                     dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)])):#[(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)] [(0.30,0.65),(0.60,0.65),(0.1,1),(1,1)]
    img_size = np.float32([(img.shape[1],img.shape[0])])
    src = src* img_size
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = dst * np.float32(dst_size)
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, dst_size,cv2.INTER_LINEAR)
    return warped

In [4]:
def all_in_one (in_image_1):
  hls = cv2.cvtColor(in_image_1, cv2.COLOR_RGB2HLS)#.astype(np.float)
  l_channel = hls[:,:,1]  
  s_channel = hls[:,:,2]
  h_channel = hls[:,:,0]
  
  grad_x = abs_sobel_thresh(l_channel, 'x', thresh=(10, 200))
  dir_binary = dir_threshold(l_channel, thresh=(np.pi/6, np.pi/2))
  combined = ((grad_x == 1) & (dir_binary == 1))
  combined_binary = np.array(combined).astype(np.uint8)

  after_warp = perspective_warp(combined_binary,
                                dst_size=(1280,720),
                                src=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]),
                                dst=np.float32([(0,0), (1, 0), (0,1), (1,1)]))

  frame, left_curverad, right_curverad = search_around_poly(after_warp)   #Final_warp)
  #frame after inverse prespective
  frame =inv_perspective_warp(frame, 
                      dst_size=(in_image_1.shape[1],in_image_1.shape[0]),
                      src=np.float32([(0,0), (1, 0), (0,1), (1,1)]),
                      dst=np.float32([(0.43,0.65),(0.58,0.65),(0.1,1),(1,1)]))
  frame = cv2.addWeighted(frame, 0.3, in_image_1, 0.7, 0) 
  #Add curvature and distance from the center
  curvature = (left_curverad + right_curverad) / 2     # get radius of left_lane & radius of right_lane /2 -->to get radius of curvature located at middle of lane
  print(left_curverad)
  print(right_curverad)
  print(curvature)    #radius of curvature  print()
  car_pos = in_image_1.shape[1] / 2  #middle point of the image
  center = (abs(car_pos - curvature)*(3.7/650))/10  #distance from the center  
  curvature = 'Radius of Curvature: ' + str(round(curvature, 2)) + 'm'
  center = str(round(center, 3)) + 'm away from center'
  frame = cv2.putText(frame, curvature, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
  frame = cv2.putText(frame, center, (50, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
  return frame